## 7. Cluster_DatosFull

### Objetivo

Generar cluster's de estaciones en base sólo a la variable de ratio de demanda histórica

### Descripción General de notebook

    1. Carga de datos base proveniente de etapa de preparación
    2. Clusterización final en base a k=5

## 1. Carga de datos de entrada

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.cluster as cluster
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
import sklearn.cluster as cluster
import sklearn.metrics as metrics
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

%matplotlib inline

pd.set_option('display.max_rows', 5)

In [2]:
df_FinalCluster_Original = pd.read_csv('df_FinalCluster.csv',
                              dtype={'Distrito':'category','Plazas':'category','Barrio':'category'})

df_FinalCluster_Original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ESTACION             179 non-null    int64   
 1   DEMANDA_RATIO        179 non-null    float64 
 2   Gis_X                179 non-null    float64 
 3   Gis_Y                179 non-null    float64 
 4   Distrito             179 non-null    category
 5   Barrio               179 non-null    category
 6   Plazas               179 non-null    category
 7   Longitud             179 non-null    float64 
 8   Latitud              179 non-null    float64 
 9   Tasa_Paro            179 non-null    float64 
 10  Renta_Media_Persona  179 non-null    float64 
 11  Renta_Media_Hogar    179 non-null    float64 
 12  Poblacion            179 non-null    float64 
dtypes: category(3), float64(9), int64(1)
memory usage: 16.7 KB


### Selección de Variables DEMANDA RATIO

In [3]:
df_FinalCluster = df_FinalCluster_Original.copy()

df_FinalCluster = df_FinalCluster.drop(columns='ESTACION')

df_FinalCluster = df_FinalCluster[['DEMANDA_RATIO']]

df_FinalCluster.info()

num_cols = list(df_FinalCluster._get_numeric_data().columns)
cat_cols = list(set(df_FinalCluster.columns) - set(num_cols))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DEMANDA_RATIO  179 non-null    float64
dtypes: float64(1)
memory usage: 1.5 KB


In [4]:
df_FinalCluster

,DEMANDA_RATIO
0,2450.230769
1,1273.636364
2,2010.576923
3,1351.346154
4,1242.769231
...,...
174,698.000000
175,841.750000
176,1019.571429
177,868.285714


In [5]:
print(num_cols)
print(cat_cols)

['DEMANDA_RATIO']
[]


### Concatenación de DF: Numéricos y Categóricas

In [7]:
df_knn_final = df_FinalCluster[num_cols].copy()
df_knn_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DEMANDA_RATIO  179 non-null    float64
dtypes: float64(1)
memory usage: 1.5 KB


## 2. Clusterización final en base a k=5

La clusterización se ejecuta para cada DataFrame según PCA siempre considerando k=5. 
La ejecución del algoritmo final considera:

    - Estandarización: MinMaxScaler()
    - Clusterización: KMeans según parámetros observados en siguiente celda

In [8]:
n_cluster = 5

scaler = MinMaxScaler()
kmeans = KMeans(
               n_clusters=n_cluster,
               init="k-means++",
               n_init=50,
               max_iter=500,
               random_state=200,)

pipeline = make_pipeline(scaler, kmeans)
pipeline

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kmeans',
                 KMeans(max_iter=500, n_clusters=5, n_init=50,
                        random_state=200))])

In [9]:
df_pred_knn_final_t = pipeline.fit_transform(df_knn_final)
df_pred_knn_final = pipeline.fit_predict(df_knn_final)


In [10]:
df_pred_knn_final_t

array([[5.92639872e-02, 2.55814107e-01, 4.10845694e-01, 3.47084849e-01,
        1.17381776e-01],
       [3.72813462e-01, 5.77353680e-02, 9.72962183e-02, 6.60634325e-01,
        1.96167700e-01],
       [1.76426904e-01, 1.38651190e-01, 2.93682776e-01, 4.64247766e-01,
        2.18858619e-04],
       [3.52104658e-01, 3.70265631e-02, 1.18005023e-01, 6.39925520e-01,
        1.75458895e-01],
       [3.81039214e-01, 6.59611196e-02, 8.90704667e-02, 6.68860076e-01,
        2.04393451e-01],
       [3.54029714e-02, 2.79675123e-01, 4.34706709e-01, 3.23223834e-01,
        1.41242791e-01],
       [3.06832275e-01, 8.24581937e-03, 1.63277406e-01, 5.94653137e-01,
        1.30186512e-01],
       [2.49311648e-01, 6.57664466e-02, 2.20798033e-01, 5.37132510e-01,
        7.26658849e-02],
       [6.46841500e-02, 3.79762244e-01, 5.34793831e-01, 2.23136712e-01,
        2.41329913e-01],
       [2.98222630e-01, 1.68554642e-02, 1.71887051e-01, 5.86043492e-01,
        1.21576867e-01],
       [4.17865945e-01, 1.0278

In [11]:
df_pred_knn_final

array([0, 1, 4, 1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 2, 1, 4, 1, 0, 1, 1, 1,
       4, 1, 2, 2, 4, 0, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0, 4, 3, 1, 0, 0, 1,
       4, 0, 1, 1, 4, 4, 1, 4, 0, 3, 0, 0, 2, 2, 4, 1, 0, 4, 1, 1, 1, 4,
       2, 1, 4, 4, 4, 1, 4, 4, 1, 1, 1, 0, 0, 2, 4, 2, 2, 1, 3, 4, 1, 1,
       1, 4, 1, 2, 2, 1, 2, 2, 1, 4, 2, 2, 1, 2, 4, 2, 1, 2, 2, 4, 4, 1,
       1, 2, 4, 2, 2, 1, 2, 1, 1, 1, 1, 2, 0, 0, 1, 4, 4, 4, 1, 0, 4, 2,
       2, 4, 2, 2, 1, 2, 2, 4, 2, 2, 2, 3, 2, 2, 2, 1, 1, 4, 4, 0, 2, 2,
       0, 0, 0, 3, 0, 2, 4, 1, 0, 0, 0, 1, 1, 2, 2, 3, 1, 2, 2, 2, 2, 2,
       2, 2, 2])

In [12]:
df_final_Clusterizado = pd.concat([df_FinalCluster_Original, pd.DataFrame(df_pred_knn_final)], axis=1)

df_final_Clusterizado.rename(columns={0: 'CLUSTER'}, inplace=True)
df_final_Clusterizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ESTACION             179 non-null    int64   
 1   DEMANDA_RATIO        179 non-null    float64 
 2   Gis_X                179 non-null    float64 
 3   Gis_Y                179 non-null    float64 
 4   Distrito             179 non-null    category
 5   Barrio               179 non-null    category
 6   Plazas               179 non-null    category
 7   Longitud             179 non-null    float64 
 8   Latitud              179 non-null    float64 
 9   Tasa_Paro            179 non-null    float64 
 10  Renta_Media_Persona  179 non-null    float64 
 11  Renta_Media_Hogar    179 non-null    float64 
 12  Poblacion            179 non-null    float64 
 13  CLUSTER              179 non-null    int32   
dtypes: category(3), float64(9), int32(1), int64(1)
memory usage: 17.4 KB


In [14]:
df_final_Clusterizado

,ESTACION,DEMANDA_RATIO,Gis_X,Gis_Y,Distrito,Barrio,Plazas,Longitud,Latitud,Tasa_Paro,Renta_Media_Persona,Renta_Media_Hogar,Poblacion,CLUSTER
0,1,2450.230769,440443.61,4474290.65,1,01-06,30,-3.701998,40.417111,4.309681,17550.00279,35056.88536,7665.0,0
1,2,1273.636364,440480.56,4474301.74,1,01-06,30,-3.701564,40.417213,4.309681,17550.00279,35056.88536,7665.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,199,868.285714,442830.66,4476215.36,4,04-04,18,-3.674037,40.434617,3.819713,21251.03416,50063.95065,42445.0,2
178,215,693.857143,440874.02,4479384.51,6,06-05,18,-3.697399,40.463029,5.899887,12254.49279,30163.55029,26607.0,2


In [13]:
df_final_Clusterizado.to_csv('../../data/Clusterizada_soloDemandaRatio.csv', index=False)